In [101]:
# Ubicaciones y tipo de archivo
file_comic = "/home/adrian/Documentos/AEA/examen_diagnostico_datio/src/main/resources/input/csv/comics/comics.csv"
file_charac_comic = "/home/adrian/Documentos/AEA/examen_diagnostico_datio/src/main/resources/input/csv/comics/charactersToComics.csv"
file_charac = "/home/adrian/Documentos/AEA/examen_diagnostico_datio/src/main/resources/input/csv/comics/characters.csv"
file_type = "csv"

# Propiedasdes del CSV generales
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

#Cargar Comics
comics = spark.read.format(file_type) \
.option('inferSchema',infer_schema) \
.option('header', first_row_is_header) \
.option('sep', delimiter) \
.load(file_comic)

#Cargar relacion comics_characters
charac_comic = spark.read.format(file_type) \
.option('inferSchema',infer_schema) \
.option('header', first_row_is_header) \
.option('sep', delimiter) \
.load(file_charac_comic)

#Cargar characters
charac = spark.read.format(file_type) \
.option('inferSchema',infer_schema) \
.option('header', first_row_is_header) \
.option('sep', delimiter) \
.load(file_charac)

#comics.show()
#charac_comic.show()
#charac.show()


In [ ]:
from pyspark.sql import functions as F

#inner join
comic_full = comics.join(charac_comic, comics.comicID == charac_comic.comicID, how='inner').join(charac, charac_comic.characterID == charac.characterID, how='inner')

nuevo = comic_full.groupby(comics.comicID, comics.title, comics.issueNumber, comics.description).agg(F.collect_list(charac.name).alias("personajes"))

nuevo.write.parquet("/home/adrian/Documentos/AEA/examen_diagnostico_datio/src/main/output/parquet/comics.parquet")